# Clapton

Read the CAFQA notebook first to understand the basic objects!

In [17]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize
from clapton.ansatzes import circular_ansatz
from clapton.depolarization import GateGeneralDepolarizationModel
from clapton.evaluation import transform_paulis
import numpy as np

In [2]:
# define Hamiltonian, e.g. 3q Heisenberg model with random coefficients
paulis = ["XXI", "IXX", "YYI", "IYY", "ZZI", "IZZ"]
coeffs = np.random.random(len(paulis))

In [3]:
# define empty VQE Clifford circuit that captures noise but essentially acts as
# identity
# here we use the circular_ansatz template
# we fix 2q gates
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)

In [7]:
# let's add a noise model where we specify global 1q and 2q gate errors
nm = GateGeneralDepolarizationModel(p1=0.005, p2=0.02)
vqe_pcirc.add_depolarization_model(nm)

In [8]:
# we can look at the corresponding stim circuit
vqe_pcirc.stim_circuit().diagram()

q0: -I-DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.02)-@-DEPOLARIZE2(0.02)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------
                                                                                                                         | |                 | |
q1: ---I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------|-|-----------------X-DEPOLARIZE2(0.02)-@-DEPOLARIZE2(0.02)-I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------
                                                                                                                         | |                                     | |
q2: ----------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-@-DEPOLARIZE2(0.02)---------------------X-DEPOLARIZE2(0.02)--------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-

In [9]:
# we need a circuit that parametrizes the Clapton transformation
# also use circular ansatz (could be different) but allow optimization over 2q gates
trans_pcirc = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=False)

In [13]:
# we can perform Clapton by using the main optimization function "claptonize"
# vqe_pcirc will stay fixed, optimization happens over trans_pcirc
ks_best, energy_noisy, energy_noiseless = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    trans_pcirc,        # specify transformation circuit
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id 1 with 1 procs
started GA at id 2 with 1 procs


started GA at id 3 with 1 procs



/local/lmseifert/conda_envs/clapton/lib/python3.12/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


started GA at id None with 1 procs



/local/lmseifert/conda_envs/clapton/lib/python3.12/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
/local/lmseifert/conda_envs/clapton/lib/python3.12/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution a

[0, array([-1.6408493 , -0.78597491, -0.85487439,  0.        ]), array([0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 2, 1, 0, 2, 2], dtype=object)][0, array([-1.64757246, -0.79269807, -0.85487439,  0.        ]), array([2, 0, 3, 2, 3, 0, 0, 0, 0, 2, 2, 1, 0, 3, 3], dtype=object)][0, array([-1.64555628, -0.79068189, -0.85487439,  0.        ]), array([2, 0, 3, 1, 2, 0, 0, 0, 0, 2, 2, 1, 0, 3, 2], dtype=object)]


[0, array([-1.64323681, -0.78836243, -0.85487439,  0.        ]), array([1, 2, 2, 0, 2, 0, 0, 0, 0, 3, 0, 2, 1, 0, 3], dtype=object)]
[1, array([-1.6354825 , -0.78060811, -0.85487439,  0.        ]), array([2, 2, 1, 1, 1, 2, 0, 0, 0, 2, 0, 1, 1, 0, 3], dtype=object)][1, array([-1.24684523, -0.59970423, -0.647141  ,  0.        ]), array([2, 3, 1, 1, 2, 0, 0, 0, 0, 3, 3, 1, 0, 2, 1], dtype=object)]

[1, array([-1.64859424, -0.79371985, -0.85487439,  0.        ]), array([0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 2, 2, 2, 3, 3], dtype=object)]
[1, array([-1.64423049, -0.7893561 , -0.85487439,  0.        ]), arra

In [14]:
# the best parameters are
ks_best

[0, 2, 3, 1, 0, 1, 0, 1, 2, 2, 0, 0, 2, 3, 2]

In [15]:
# with noisy/noiseless energies
energy_noisy, energy_noiseless

(np.float64(-0.7861735299700903), np.float64(-0.8700453771484478))

In [16]:
# the transformation circuit is
trans_pcirc.assign(ks_best)
trans_pcirc.stim_circuit().diagram()

q0: -I----------S-@-Y-Z-------
                  |
q1: -Y----------I-X-X-I-S_DAG-
                    |
q2: -SQRT_Y_DAG-S---@-I-Z-----

In [22]:
# the transformed Hamiltonian is given by
paulis_new, signs = transform_paulis(trans_pcirc.stim_circuit(), paulis, replace_I=True)
coeffs_new = np.multiply(coeffs, signs)
paulis_new, coeffs_new

(['XZX', 'ZZZ', 'XII', 'IIZ', 'IZX', 'ZZI'],
 array([-0.03791464, -0.14899336,  0.26810406, -0.07391102,  0.20773338,
        -0.647141  ]))